In [8]:
import numpy as np
import matplotlib.pyplot as plt

from utils import smooth, system_matrix_1d

In [9]:
def restrict(x, sparse=False):
    """Applies the restriction operator to vector x. 

    Parameters
    ----------
    x : numpy.array
        vector on finer grid

    Returns
    ------- 
    numpy.array
        vector on coarse grid
    """
    n = x.shape[0]
    nr = int((n-1)/2)
    R = np.zeros((nr, n))
    R[:,0] = 0.25
    R[:,1] = 0.50
    R[:,2] = 0.25

    # Roll rows of R
    r = np.arange(0, 2*R.shape[0], 2)
    rows, column_indices = np.ogrid[:R.shape[0], :R.shape[1]]
    r[r < 0] += R.shape[1]
    column_indices = column_indices - r[:, np.newaxis]
    R = R[rows, column_indices]
    x_restricted =  np.matmul(R, x)

    return x_restricted

In [10]:
def interpolate(x, sparse=False):
    """Applies the interpolation operator to vector x

    Parameters
    ----------
    x : numpy.array
        vector on coarse grid

    Returns
    ------- 
    numpy.array
        vector on fine grid
    """
    # Pad with zeros
    x_padded = np.concatenate(([0], x, [0]))
    ni = 2*x.shape[0]+1
    I = np.zeros((ni, x_padded.shape[0]))
    I[::2,0:2] = [0.5, 0.5]
    I[1::2,0] = 1

    # Roll rows of I
    r = np.zeros(I.shape[0]+1)
    r[::2] = np.arange((r.shape[0])/2)
    r[1::2] = np.arange(r.shape[0]/2)
    r = r[1:]

    for k in range(r.shape[0]):
        I[k,:] = np.roll(I[k,:], int(r[k]))

    return np.matmul(I, x_padded)

In [11]:
def P(i, f):
    """Generate the problem for 1-dimensional discrete Poisson equation

    Parameters
    ----------
    i : int
        Number of unknowns will be 2**i - 1
        
    f : function
        The load function

    Returns
    -------
    numpy.array
        System matrix  
    numpy.array
        right-hand side
    numpy.array
        grid
    """
    number_of_points = 2**i + 1
    t = np.linspace(0, 1, number_of_points)
    T = system_matrix_1d(i)
    b = f(t[1:-1])
    return T, b, t[1:-1]

In [50]:
# Multigrid V-cycle
def MGV(b, x, i):
    """Recursive Multigrid V-cycle algorithm

    Parameters
    ----------
    b_i : numpy.array
        right-hand side of problem P(i), fine grid
        
    x_i : numpy.array
        approximate solution of problem P(i), fine grid

    Returns
    -------
    numpy.array
        solution to the problem P(i)
    """
    T = system_matrix_1d(i)
    
    if i == 1:
        print("MGV called with i = 1")
        x = np.linalg.solve(T, b)
        return x
    else:
        print("MGV called with i = ", i)
        # Smoothen the approximate solution
        x = smooth(x=x, A=T, b=b)
        
        # Calculate residual
        r = np.dot(T, x) - b
        
        # Restrict the residual to coarser grid
        r = restrict(x=r)
        
        # Solve the correction term d, initial guess at 0
        x0 = np.zeros_like(r)
        d = MGV(b=r, x=x0, i=i-1)
        
        # Interpolate to finer grid
        d = interpolate(d)
        
        # Make the correction and smoothen some more
        x = x - d
        x = smooth(x=x, A=T, b=b)
        return x
    

# Full Multigrid
def FMG(f, m):
    T, b, t = P(1, f)
    x = np.linalg.solve(T, b)
    xs = []
    bs = []
    ts = []
    xs.append(x)
    bs.append(b)
    ts.append(t)
    for i in range(2, m):
        x_in = interpolate(x)
        _, _, t_in = P(i, f)
        b_in = f(t_in)
        x = MGV(b_in, x_in, i)
        xs.append(x)
        bs.append(b_in)
        ts.append(t_in)
    return xs, bs, ts

In [51]:
FMG(f, 5)

MGV called with i =  2
MGV called with i = 1
MGV called with i =  3
MGV called with i =  2
MGV called with i = 1
MGV called with i =  4
MGV called with i =  3
MGV called with i =  2
MGV called with i = 1


([array([1.]),
  array([0.75, 1.  , 0.75]),
  array([0.43634259, 0.75462963, 0.94097222, 1.01003086, 0.94097222,
         0.75462963, 0.43634259]),
  array([0.23379272, 0.43843355, 0.61045025, 0.75295544, 0.86080747,
         0.94104605, 0.98640046, 1.00396519, 0.98640046, 0.94104605,
         0.86080747, 0.75295544, 0.61045025, 0.43843355, 0.23379272])],
 [array([8.]),
  array([8., 8., 8.]),
  array([8., 8., 8., 8., 8., 8., 8.]),
  array([8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.])],
 [array([0.5]),
  array([0.25, 0.5 , 0.75]),
  array([0.125, 0.25 , 0.375, 0.5  , 0.625, 0.75 , 0.875]),
  array([0.0625, 0.125 , 0.1875, 0.25  , 0.3125, 0.375 , 0.4375, 0.5   ,
         0.5625, 0.625 , 0.6875, 0.75  , 0.8125, 0.875 , 0.9375])])

In [52]:
def f(t):
    return np.ones_like(t) * 8

i = 5
T, b, t = P(i, f=f)
x = np.linalg.solve(T, b)
x

array([0.12109375, 0.234375  , 0.33984375, 0.4375    , 0.52734375,
       0.609375  , 0.68359375, 0.75      , 0.80859375, 0.859375  ,
       0.90234375, 0.9375    , 0.96484375, 0.984375  , 0.99609375,
       1.        , 0.99609375, 0.984375  , 0.96484375, 0.9375    ,
       0.90234375, 0.859375  , 0.80859375, 0.75      , 0.68359375,
       0.609375  , 0.52734375, 0.4375    , 0.33984375, 0.234375  ,
       0.12109375])

In [53]:
MGV(b=b, x=x, i=i)

MGV called with i =  5
MGV called with i =  4
MGV called with i =  3
MGV called with i =  2
MGV called with i = 1


array([0.12109375, 0.234375  , 0.33984375, 0.4375    , 0.52734375,
       0.609375  , 0.68359375, 0.75      , 0.80859375, 0.859375  ,
       0.90234375, 0.9375    , 0.96484375, 0.984375  , 0.99609375,
       1.        , 0.99609375, 0.984375  , 0.96484375, 0.9375    ,
       0.90234375, 0.859375  , 0.80859375, 0.75      , 0.68359375,
       0.609375  , 0.52734375, 0.4375    , 0.33984375, 0.234375  ,
       0.12109375])

In [54]:
class PoissonProblem1D:
    def __init__(self, f):
        self.f = f
        self.u = lambda t: -4*t**2+4*t
        
    def solve_fmg(self, m):
        self.xs, self.bs, self.ts = FMG(f=self.f, m=m)

In [55]:
poisson = PoissonProblem1D(f=f)

In [56]:
poisson.solve_fmg(6)

MGV called with i =  2
MGV called with i = 1
MGV called with i =  3
MGV called with i =  2
MGV called with i = 1
MGV called with i =  4
MGV called with i =  3
MGV called with i =  2
MGV called with i = 1
MGV called with i =  5
MGV called with i =  4
MGV called with i =  3
MGV called with i =  2
MGV called with i = 1


# Solutions

In [57]:
poisson.xs

[array([1.]),
 array([0.75, 1.  , 0.75]),
 array([0.43634259, 0.75462963, 0.94097222, 1.01003086, 0.94097222,
        0.75462963, 0.43634259]),
 array([0.23379272, 0.43843355, 0.61045025, 0.75295544, 0.86080747,
        0.94104605, 0.98640046, 1.00396519, 0.98640046, 0.94104605,
        0.86080747, 0.75295544, 0.61045025, 0.43843355, 0.23379272]),
 array([0.12090257, 0.23451564, 0.34001139, 0.43813805, 0.52764137,
        0.61024157, 0.68411331, 0.75098916, 0.80912166, 0.86035972,
        0.90295066, 0.93859331, 0.96552156, 0.98550892, 0.99682029,
        1.00118972, 0.99682029, 0.98550892, 0.96552156, 0.93859331,
        0.90295066, 0.86035972, 0.80912166, 0.75098916, 0.68411331,
        0.61024157, 0.52764137, 0.43813805, 0.34001139, 0.23451564,
        0.12090257])]

# Grids

In [60]:
poisson.ts

[array([0.5]),
 array([0.25, 0.5 , 0.75]),
 array([0.125, 0.25 , 0.375, 0.5  , 0.625, 0.75 , 0.875]),
 array([0.0625, 0.125 , 0.1875, 0.25  , 0.3125, 0.375 , 0.4375, 0.5   ,
        0.5625, 0.625 , 0.6875, 0.75  , 0.8125, 0.875 , 0.9375]),
 array([0.03125, 0.0625 , 0.09375, 0.125  , 0.15625, 0.1875 , 0.21875,
        0.25   , 0.28125, 0.3125 , 0.34375, 0.375  , 0.40625, 0.4375 ,
        0.46875, 0.5    , 0.53125, 0.5625 , 0.59375, 0.625  , 0.65625,
        0.6875 , 0.71875, 0.75   , 0.78125, 0.8125 , 0.84375, 0.875  ,
        0.90625, 0.9375 , 0.96875])]